# Selenium 이용한 뉴스 기사 개요와 댓글 scraping

- Selenium document
https://selenium-python.readthedocs.io/locating-elements.html

In [1]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

In [2]:
driver = webdriver.Chrome('chromedriver')   # Selenium

In [19]:
Article_df = pd.DataFrame(columns=['news_site', 'post_id', 'url', 'date', 'time', 'title', 'writer',  
                                   'cmt_count'])
Comment_df = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 'user_name',
                                   'user_id','yes', 'no', 'cmt_text'])

Article_df_temp = pd.DataFrame(columns=['news_site', 'post_id', 'url', 'date', 'time', 'title', 'writer',  
                                   'cmt_count'])
Comment_df_temp = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 'user_name',
                                   'user_id','yes', 'no', 'cmt_text'])

filename = "18_link_chosun_sample.xlsx"
df_url_list = pd.read_excel(filename)

i = 0
while i < len(df_url_list):
    post_id = df_url_list.iloc[i].post_id
    url = df_url_list.iloc[i].link
    print("processing: ", url)
    i += 1
    if 'chosun' in url: 
        try:
            driver.get(url)   
        except:
            print("----처리 불가능한 site\n")
            continue

        news_site='chosun'
        Article_df_temp, Comment_df_temp  = chosun(url, post_id) 
        Article_df = Article_df.append(Article_df_temp)
        Comment_df = Comment_df.append(Comment_df_temp)
        print("----Done\n")

writer = pd.ExcelWriter('article.xlsx', engine='xlsxwriter')
Article_df.to_excel(writer)
writer.save() 

writer = pd.ExcelWriter('comment.xlsx', engine='xlsxwriter')
Comment_df.to_excel(writer)
writer.save() 

processing:  http://news.chosun.com/site/data/html_dir/2017/09/10/2017091002294.html?fbclid=IwAR3WeBo4jYHtFZrTnPckLDDPJccC-DzS3rLl8Ryzfo8H6WO2l0sYwzRFufU
1 ['안석배', '논설위원']
----Done

processing:  https://biz.chosun.com/site/data/html_dir/2019/11/17/2019111700344.html
2 ['오광진', '정보과학부장']
----Done

processing:  http://news.chosun.com/site/data/html_dir/2019/10/25/2019102500045.html
1 ['곽수근', '기자']
----Done



In [ ]:
def joins(url, FB_post_id):
    #make DataFrame
    my_computer = 1 #컴퓨터 사양에 따라 조절
    
    Article_df = pd.DataFrame(columns=['news_c', 'FB_post_id', 'url', 'date', 'time', 'title', 'writer', 'text', 'cmt_count', 'scraped_time'])
    Comment_df = pd.DataFrame(columns=['FB_post_id', 'cmt_id', 're_reply', 'rmt_count', 'rmt_id', 'cmt_date', 'cmt_time', 'user_name','user_id','yes', 'no', 'cmt_text', 'scraped_time'])

    #더보기 button
    #time.sleep(my_computer)
    while True:
        try:
            element = driver.find_element_by_class_name("comment_area")
            element_comment = element.find_element_by_class_name("ft")
            element_next=element_comment.find_element_by_class_name("icon")
            element_next.click()
            time.sleep(my_computer)
        except:
            break
    time.sleep(my_computer)

    #parsed html
    html = driver.page_source
    parsed = BeautifulSoup(html, "html.parser")

    ## 기사 본문
   
    #기사 title
    try:
        title = parsed.find('h1', {'id': 'article_title'}).text
    except:
        title = "null"

    #기사 text
    try:
        text = " ".join(parsed.find('div', {'id':"article_body"}).text.split())
    except:
        text = "null"

    #기사 입력시간
    try:
        enter = parsed.find('div', {'class':"byline"}).findAll('em')[1].text.split(" ")
        a_date = enter[1]
        a_time = enter[2]
    except:
        a_date = "null"
        a_time = "null"

    #기자, 작성자
    try:
        writer_info = parsed.find('strong', {'class':"mg"}).text.split(' ')
        writer = writer_info[0]
    except:
        writer = "null"

    #총 댓글 수 파악
    try:
        cmt_count_written = parsed.find('div', {'class': 'comment_area'}).find('h3',{'class': 'subtit_comm'}).find('span').text
    except:
        cmt_count_written = 0
        
    #print("총 댓글 수: ",cmt_n)

    # 기사 DF 추가
    Article_df = Article_df.append({'news_c' : 'joins', 'FB_post_id': FB_post_id, 'url': url, 'date': a_date, 'time': a_time, 
                                    'title': title, 'writer': writer, 'text': text, 'cmt_count':cmt_count_written, 
                                    'scraped_time':datetime.datetime.now()}, ignore_index = True)

    ## joins comment scraping
    try:
        comments = parsed.findAll('li', {'class': 'item'})
        cmt_count=len(comments) #일반 댓글 수
    except:
        cmt_count=0 #일반 댓글 수
        
    cmt_no = 0
    total_rereply_count=0 #대댓글 수
    for cmt in comments:
        cmt_no +=1
        cmt_id = ('%s_C%s' %(FB_post_id, cmt_no))
        
        try:
            rmt_count=len(cmt.find('div',{'class':'reply_area'}).findAll('div',{'class':'cmt_area'}))
        except:
            rmt_count=0
        total_rereply_count += rmt_count
        yes = 0
        no = 0
        try:
            date = cmt.find('div',{'class':'cmt_wrap'}).find('span',{'class':'date'}).text.split(" ")[0]
            time_ = cmt.find('div',{'class':'cmt_wrap'}).find('span',{'class':'date'}).text.split(" ")[1]
            user_id = cmt.find('div',{'class':'cmt_wrap'}).find('strong').text
            text = cmt.find('div',{'class':'cmt_wrap'}).find('p',{'class':'content'}).text
            yes = cmt.find('div',{'class':'cmt_wrap'}).find('em',{'class':'good'}).text
            no = cmt.find('div',{'class':'cmt_wrap'}).find('em',{'class':'bad'}).text
            Comment_df = Comment_df.append({'FB_post_id': FB_post_id, 'cmt_id': cmt_id, 're_reply': "No",'rmt_count': rmt_count, 
                                            'rmt_id': "", 'cmt_date': date, 'cmt_time': time_, 'user_name': "null", 'user_id': user_id, 
                                            'yes': yes, 'no': no, 'cmt_text': text, 'scraped_time':datetime.datetime.now()}, ignore_index = True)
        except:
            Comment_df = Comment_df.append({'FB_post_id': FB_post_id, 'cmt_id': cmt_id, 're_reply': "No", 'rmt_count': rmt_count,
                                            'rmt_id': "", 'cmt_date': "null", 'cmt_time': "null", 'user_name': "null",'user_id': "null",
                                            'yes': "null", 'no': "null", 'cmt_text': "삭제된글", 'scraped_ime': datetime.datetime.now()}, ignore_index = True)
        rmt_id = 0
        if rmt_count != 0:
            for i in range(0,rmt_count):
                rmt_id = i+1
                cmt_id = ('%s_C%s_R%s' %(FB_post_id, cmt_no, rmt_id))
                try:
                    date = cmt.find('div',{'class':'reply_area'}).findAll('span',{'class':'date'})[i].text.split(" ")[0]
                    time_ = cmt.find('div',{'class':'reply_area'}).findAll('span',{'class':'date'})[i].text.split(" ")[1]
                    user_id = cmt.find('div',{'class':'reply_area'}).findAll('strong')[i].text
                    text = cmt.find('div',{'class':'reply_area'}).findAll('p',{'class':'content'})[i].text
                    yes = cmt.find('div',{'class':'reply_area'}).findAll('em',{'class':'good'})[i].text
                    no = cmt.find('div',{'class':'reply_area'}).findAll('em',{'class':'bad'})[i].text
                    Comment_df = Comment_df.append({'FB_post_id': FB_post_id, 'cmt_id': cmt_id, 're_reply': "Yes",'rmt_id': rmt_id, 
                                                    'cmt_date': date, 'cmt_time': time_, 'user_name': "null", 'user_id': user_id, 
                                                    'yes': yes, 'no': no, 'cmt_text': text, 'scraped_time':datetime.datetime.now()}, ignore_index = True)
                except:
                    Comment_df = Comment_df.append({'FB_post_id': FB_post_id, 'cmt_id': cmt_id, 're_reply': "Yes", 'rmt_id': "null", 
                                                    'cmt_date': "null", 'cmt_time': "null", 'user_name': "null",'user_id': "null",
                                                    'yes': "null", 'no': "null", 'cmt_text': "삭제된글", 'scraped_time': datetime.datetime.now()}, ignore_index = True)

    #joing는 표시되는 댓글 수가 대댓글 포함
    if cmt_count_written == 0 and int(cmt_count_written) == (int(cmt_count)+int(total_rereply_count)):
        with open(log_file_name, 'a') as f:
            f.write("----댓글 없음\n")
    elif int(cmt_count_written) != ((int(cmt_count)+int(total_rereply_count))):
        with open(log_file_name, 'a') as f:
            f.write("----적혀진 댓글 %s, 수집된 댓글 %s, 수집된 대댓글 %s\n" %(cmt_count_written,cmt_count, total_rereply_count))
            f.write("----[Error] 댓글 수와 수집된 댓글 수가 일치하지 않음\n")
    else:
        with open(log_file_name, 'a') as f:
            f.write("----적혀진 댓글 %s, 수집된 댓글 %s, 수집된 대댓글 %s\n" %(cmt_count_written,cmt_count, total_rereply_count))
            f.write("----댓글 수집 완료\n")
            
    return Article_df, Comment_df

In [18]:
def chosun(url, post_id):
    #make DataFrame
    Article_df = pd.DataFrame(columns=['news_site', 'post_id','url', 'date', 'time', 'title', 'writer', 'cmt_count'])
    Comment_df = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 
                                       'user_name','user_id','yes', 'no', 'cmt_text'])

    #parsed html
    html = driver.page_source
    parsed = BeautifulSoup(html, "html.parser")

    ## 기사 본문
    #기사 title
    try: 
        title = parsed.find('div', {'class': "news_title_text"}).text
    except:
        title = "null"

    #기사 입력시간
    try:
        date_time = parsed.find('div', {'class':"news_date"}).text.split('|')[0].strip().split(" ")
        date = date_time[1]
        time = date_time[2]
    except:
        date = "null"
        time = "null"
        
    #기자, 작성자
    writer = "null"
    try:
        writer_info = parsed.find('li', {'class':"author"}).text.strip().split(" ")
        print('1', writer_info)
        writer = writer_info[0]
    except:
        try:
            writer_info = parsed.find('li', {'id':"j1"}).text.strip().split(" ")
            writer = writer_info[0]
            print('2', writer_info)
        except:
            writer = "null"

    # 총 댓글 수 파악
    try:
        cmt_count_written = parsed.find('em', {'class': 'cmt_count'}).text
    except:
        cmt_count_written = 0

    # 기사 DF 추가
    Article_df = Article_df.append({'news_site' : 'chosun', 'post_id': post_id, 'url': url, 
                                    'date': date, 'time': time, 'title': title, 'writer': writer, 
                                    'cmt_count':cmt_count_written}, ignore_index = True)

    #move comment page (chosun)
    try:
        element_comment = driver.find_element_by_class_name("cmt_count")   #Selenium
        element_comment.click()                                            #Selenium
    except:
        pass

    ## chosun comment scraping
    # 첫번째 page
    page_n = 1

    #parsed html
    html = driver.page_source
    parsed = BeautifulSoup(html, "html.parser")

    try:
        comments = parsed.findAll('div', {'class': 'cmt_box'})
        cmt_count=len(comments) #일반 댓글 수
       
    except:
        cmt_count = 0

    cmt_no = 0
    for cmt in comments:
        cmt_no += 1
        cmt_id = ('%s_C%s' %(post_id, cmt_no))
        
        yes = 0
        no = 0
        
        try:
            cmt_date = cmt.find('dl',{'class':'cmt_comment'}).find('span', {'class': 'cmt_date'}).text
            cmt_time = cmt.find('dl',{'class':'cmt_comment'}).find('span', {'class': 'cmt_time'}).text
            user_name = cmt.find('dl',{'class':'cmt_comment'}).find('a').text
            user_id = cmt.find('dl',{'class':'cmt_comment'}).find('sapn',{'class', 'cmt_user_id'}).text
            text = cmt.find('dl',{'class':'cmt_comment'}).find('dd', {'class': 'cmt_txt'}).text
            yes = cmt.find('dl',{'class':'cmt_comment'}).find('a', {'class': 'cmt_reply_yes'}).find('label').text
            no = cmt.find('dl',{'class':'cmt_comment'}).find('a', {'class': 'cmt_reply_no'}).find('label').text
            
            Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id,  
                                            'cmt_date': cmt_date, 'cmt_time': cmt_time, 
                                            'user_name': user_name, 'user_id': user_id, 'yes': yes, 'no': no, 
                                            'cmt_text': text}, ignore_index = True)
        except:
            Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id, 
                                            'cmt_date': "null", 'cmt_time': "null", 
                                            'user_name': "null",'user_id': "null",'yes': "null", 'no': "null", 
                                            'cmt_text': "삭제된글"}, ignore_index = True)
        
    # 넘어가야 할 페이지가 더 있는지 확인    
    try:
        pages = parsed.find('ul', {'class': 'paginate_num'})
        num_a = pages.findAll('a')
        n = len(num_a) - 1     # 이전, 현재, 다음도 a로 묶여 있으므로 이들 제외하고 넘어가야 하는 페이지 수
        page_links = [pages.findAll('a')[i]['href'] for i in range(2, n)]  # 세번째 a, 즉 [2]부터 link가 있게 됨
        #print(page_links)
    except:
        page_links = []

    # 있다면 이후 작업
    for page in page_links:
        page_n += 1

        driver.get(page)
        html = driver.page_source

        parsed = BeautifulSoup(html, "html.parser")  
        comments = parsed.findAll('div', {'class': 'cmt_box'})
        cmt_count += len(comments)

        #print(comments)

        #print('%d 페이지 댓글 수: %d' %(page_n, len(comments)))

        for cmt in comments:
            cmt_no += 1
            cmt_id = ('%s_C%s' %(post_id, cmt_no))
            yes = 0
            no = 0
            try:
                cmt_date = cmt.find('dl',{'class':'cmt_comment'}).find('span', {'class': 'cmt_date'}).text
                cmt_time_ = cmt.find('dl',{'class':'cmt_comment'}).find('span', {'class': 'cmt_time'}).text
                user_name = cmt.find('dl',{'class':'cmt_comment'}).find('a').text
                user_id = cmt.find('dl',{'class':'cmt_comment'}).find('sapn',{'class', 'cmt_user_id'}).text
                text = cmt.find('dl',{'class':'cmt_comment'}).find('dd', {'class': 'cmt_txt'}).text
                yes = cmt.find('dl',{'class':'cmt_comment'}).find('a', {'class': 'cmt_reply_yes'}).find('label').text
                no = cmt.find('dl',{'class':'cmt_comment'}).find('a', {'class': 'cmt_reply_no'}).find('label').text
                Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id, 
                                            'cmt_date': cmt_date, 'cmt_time': cmt_time, 
                                            'user_name': user_name, 'user_id': user_id, 'yes': yes, 'no': no, 
                                            'cmt_text': text}, ignore_index = True)
            except:
                Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id,  
                                            'cmt_date': "null", 'cmt_time': "null", 
                                            'user_name': "null",'user_id': "null",'yes': "null", 'no': "null", 
                                            'cmt_text': "삭제된글"}, ignore_index = True)
    return Article_df, Comment_df

In [20]:
Article_df

,news_site,post_id,url,date,time,title,writer,cmt_count
0,chosun,chosun01,http://news.chosun.com/site/data/html_dir/2017...,2017.09.11,03:16,\n[만물상] 무릎 꿇은 장애 학생 부모\n\n조선일보\n\n\n\n\n안석배 논설...,안석배,36
0,chosun,chosun02,https://biz.chosun.com/site/data/html_dir/2019...,2019.11.17,13:05,\n금융판도 흔드는 빅테크 합종연횡\n\n\n\n조선비즈\n\n오광진 정보과학부장\...,오광진,2
0,chosun,chosun03,http://news.chosun.com/site/data/html_dir/2019...,2019.10.25,03:01,"\n""AI 과목 개설, 빅데이터 연구 대학으로 거듭날 것""\n\n조선일보\n\n\n...",곽수근,0


In [40]:
Comment_df

,post_id,cmt_id,cmt_date,cmt_time,user_name,user_id,yes,no,cmt_text
0,chosun01,chosun01_C1,2017.09.12,06:44:02,이성수,(si****),6,0,우리애들도 미국에서 명문고등학교다녔는데 나중에 보니 자페학생들이 있는걸 부모인 우리...
1,chosun01,chosun01_C2,2017.09.12,06:31:32,김진하,(shjo****),10,0,"제발 쫌 ,우리 모두를 부끄럽게 하는 이런 기사가 그만 나오기를 기도한다.누구나 장..."
2,chosun01,chosun01_C3,2017.09.12,05:32:45,김영욱,(yukon****),7,0,마지못해서 그자리에 나갔던 국개언 김성태와 집값떨어진다고 결사반대하는 지역주민들의 ...
3,chosun01,chosun01_C4,2017.09.12,03:46:27,고세호,(ela*),13,0,서울대가 장애를 이유로 불합격 시킨 수재들 전부 다 한양대에서 받아주곤 했었지. 서...
4,chosun01,chosun01_C5,2017.09.12,01:41:14,조문종,(hn****),9,0,불교도 유교도 기독교도 힘을 못쓰는 물질만능사회가 되었기 때문입니다
5,chosun01,chosun01_C6,2017.09.11,21:49:39,서석희,(ral****),20,0,무릎꿇은 부모심정... 당해보지 않으면 아무도 모를 것. 장영희를 받아준 서강대학....
6,chosun01,chosun01_C7,2017.09.11,20:20:28,송기식,(everad****),22,2,"천박한 김여사들이 항상 문제다. 그리고, 문제인도 문제다. ㅉㅉ"
7,chosun01,chosun01_C8,2017.09.11,20:15:54,서석희,(ral****),32,0,무릎꿇은 어머니들을 보고 눈물이 났다... 아파트 값 떨어진다는 천박한 이기주의자 ...
8,chosun01,chosun01_C9,2017.09.11,19:52:13,최성종,(dgjches****),22,0,"가능한 장애인과 비장애인이 구분 없이 동일한 학교에서, 같은 교실에서 교육되어야 한..."
9,chosun01,chosun01_C10,2017.09.11,19:47:56,최성종,(dgjches****),28,0,장애인 비장애인 구분 자체가 이상하다. 그럼 무엇이 정상인인가? 명확한 구분은 있는...
